# Modelo: XGBOOST

#### 1. Cargar los datos

In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
#INSERTAR CREDENCIALES
datos_completos = pd.read_csv(body)


In [2]:
datos_completos.head()
#datos_completos.info()
#datos_completos.shape #el valor de renglones debe de cambiar conforme el tiempo transcurre y se tiene más información de las ventas(3665280, 37)

,tiendaID,marcaID,skuID,gammaID,fechaID,ventas_por_mes,lag1mes_ventas_totales_por_tienda_por_mes,lag1mes_ventas_promedio_por_tienda_por_mes,lag1mes_ventas_totales_por_marca_por_mes,lag1mes_entas_promedio_por_marca_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
0,1,2,1,1,0,0,70,1.044776,355,1.047198,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
1,1,2,1,1,1,0,70,1.044776,355,1.047198,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,2,1,1,2,0,79,1.128571,379,1.046961,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
3,1,2,1,1,3,0,87,1.060976,264,1.068826,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
4,1,2,1,1,4,0,75,1.041667,136,1.079365,...,0,0.0,79,1.128571,379,1.046961,92120,1.137986,1,1.0


In [3]:
datos_modelo = datos_completos
#datos_modelo = datos_completos.drop(["punto_de_venta", "zona", "estado","ciudad","latitud","longitud"], axis=1) #en caso de que se carge el otro dataset
#datos_modelo.shape #(3665280, 30) las columnas para el modelo deben de ser 30

#### 2. Dividir en entrenamiento y prueba

In [4]:
#Hasta este número se considera el conjunto de entrenamiento
ultimo_mes_entrenamiento = datos_modelo.fechaID.unique()
ultimo_mes_entrenamiento = ultimo_mes_entrenamiento[-2]
ultimo_mes_entrenamiento

8

In [5]:
#Se tiene un mes para el conjunto de prueba
periodo_prueba = datos_modelo.fechaID.unique()
periodo_prueba = periodo_prueba[-1]
periodo_prueba

9

In [6]:
#dividir el archivo en entrenamiento y prueba
entrenamiento = datos_modelo[datos_modelo['fechaID']<=ultimo_mes_entrenamiento]
entrenamiento.head()
#entrenamiento.shape #Para este conjunto de datos, este valor va a cambiar conforme se adquieran más datos(3298752, 30)

,tiendaID,marcaID,skuID,gammaID,fechaID,ventas_por_mes,lag1mes_ventas_totales_por_tienda_por_mes,lag1mes_ventas_promedio_por_tienda_por_mes,lag1mes_ventas_totales_por_marca_por_mes,lag1mes_entas_promedio_por_marca_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
0,1,2,1,1,0,0,70,1.044776,355,1.047198,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
1,1,2,1,1,1,0,70,1.044776,355,1.047198,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
2,1,2,1,1,2,0,79,1.128571,379,1.046961,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
3,1,2,1,1,3,0,87,1.060976,264,1.068826,...,1,1.0,70,1.044776,355,1.047198,86507,1.161542,1,1.0
4,1,2,1,1,4,0,75,1.041667,136,1.079365,...,0,0.0,79,1.128571,379,1.046961,92120,1.137986,1,1.0


In [7]:
entrenamiento.shape

(7817355, 30)

In [8]:
prueba = datos_modelo[datos_modelo['fechaID']==periodo_prueba]
prueba.head()
#prueba.shape # Este número no va a cambiar dado que corresponde a la combinación de 1mes*12marcas*4gammas*4familias*1909puntosdeventa(366528, 37)

,tiendaID,marcaID,skuID,gammaID,fechaID,ventas_por_mes,lag1mes_ventas_totales_por_tienda_por_mes,lag1mes_ventas_promedio_por_tienda_por_mes,lag1mes_ventas_totales_por_marca_por_mes,lag1mes_entas_promedio_por_marca_por_mes,...,lag2mes_ventas_totales_por_sku_por_mes,lag2mes_ventas_promedio_por_sku_por_mes,lag3mes_ventas_totales_por_tienda_por_mes,lag3mes_ventas_promedio_por_tienda_por_mes,lag3mes_ventas_totales_por_marca_por_mes,lag3mes_entas_promedio_por_marca_por_mes,lag3mes_ventas_totales_por_gamma_por_mes,lag3mes_ventas_promedio_por_gamma_por_mes,lag3mes_ventas_totales_por_sku_por_mes,lag3mes_ventas_promedio_por_sku_por_mes
9,1,2,1,1,9,0,61,1.070175,53,1.081633,...,1,1.0,73,1.028169,32,1.066667,95478,1.156594,0,0.0
19,2,2,1,1,9,0,15,1.000000,53,1.081633,...,1,1.0,27,1.038462,32,1.066667,95478,1.156594,0,0.0
29,3,2,1,1,9,0,37,1.000000,53,1.081633,...,1,1.0,61,1.089286,32,1.066667,95478,1.156594,0,0.0
39,4,2,1,1,9,0,28,1.037037,53,1.081633,...,1,1.0,16,1.230769,32,1.066667,95478,1.156594,0,0.0
49,5,2,1,1,9,0,6,1.000000,53,1.081633,...,1,1.0,41,1.051282,32,1.066667,95478,1.156594,0,0.0


In [9]:
prueba.shape

(868595, 30)

#### 3. Librerías para el modelo

In [10]:
#EMPIEZA EL MODELO 
#Cargar librerías
import numpy as np 
import os
import seaborn as sns
import xgboost
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
#from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
import warnings; warnings.simplefilter('ignore')

#### 4. Preparar los datos para el modelo

In [11]:
# Separamos el conjunto de entrenamiento en y y X 
x = entrenamiento.iloc[:, entrenamiento.columns != 'ventas_por_mes'].values #arreglo de valores
y = entrenamiento.iloc[:, entrenamiento.columns == 'ventas_por_mes'].values

#### 5. Validación cruzada para series de tiempo

In [12]:
#validación cruzada: utiliza la función TimeSeriesSplit donde se parte el conjunto de entrenamiento en 3 splits de diferente longitud. 
tscv = TimeSeriesSplit(n_splits=3)#TimeSeriesSplit(max_train_size=None, n_splits=3)
my_cv = TimeSeriesSplit(n_splits=3).split(x)#<generator object TimeSeriesSplit.split at 0x7f01f9aa86d0>

In [13]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

#### 6: Aplicar un GridSearch para probar diferentes parámetros para el modelo

In [14]:
#Se prueban diferentes combinaciones de parámetros para encontrar el mejor modelo - estos valores pueden cambiarse más adelante 
params_grid = {
    'max_depth' : [2,3], #9,5,7
    'n_estimators' : [2,3],  #10
    'learning_rate' : [0.0003, 0.003], # , 0.03, 0.003, 0.0003, 0.00003
    'min_child_weight':[0.5]
}

params_fixed = {'objective':'reg:linear'}

In [15]:
# Configuración del modelo
scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error']
bst_grid = GridSearchCV(
    estimator = XGBRegressor(params_fixed, seed = 123),
    param_grid = params_grid,
    cv = my_cv,
    #n_jobs = -1, 
    scoring = scoring, 
    refit = 'neg_mean_absolute_error'
)

#### 7: Ajustar el Modelo: Entrenar el modelo con los diferentes parámetros

In [16]:
#se ajusta el modelo 2:39
bst_grid.fit(x,y)

[20:47:37] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[20:49:08] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[20:50:43] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[20:52:23] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[20:54:00] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[20:55:29] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machi

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7f9b2128f9e8>,
       error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth={'objective': 'reg:linear'}, min_child_weight=1,
       missing=None, n_estimators=100, n_jobs=1, nthread=None,
       objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=123, silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [2, 3], 'n_estimators': [2, 3], 'learning_rate': [0.0003, 0.003], 'min_child_weight': [0.5]},
       pre_dispatch='2*n_jobs', refit='neg_mean_absolute_error',
       return_train_score='warn',
       scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
       verbose=0)

#### 8: Analizar resultados de cada modelo que se corrió

In [17]:
resultados_GS = pd.DataFrame(bst_grid.cv_results_) #dataframe con resultados de todas las combinaicones de parámetros
resultados_GS

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,params,split0_test_neg_mean_squared_error,...,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_train_neg_mean_absolute_error,split1_train_neg_mean_absolute_error,split2_train_neg_mean_absolute_error,mean_train_neg_mean_absolute_error,std_train_neg_mean_absolute_error
0,22.195248,8.069531,1.898338,0.064234,0.0003,2,0.5,2,"{'learning_rate': 0.0003, 'max_depth': 2, 'min...",-0.853450,...,-0.583498,-0.528570,-0.561577,0.023755,8,-0.611338,-0.592000,-0.589169,-0.597502,0.009852
1,26.250316,9.307811,1.926429,0.053674,0.0003,2,0.5,3,"{'learning_rate': 0.0003, 'max_depth': 2, 'min...",-0.853270,...,-0.583379,-0.528446,-0.561457,0.023757,6,-0.611213,-0.591877,-0.589049,-0.597380,0.009850
2,25.700018,9.531326,1.892516,0.036140,0.0003,3,0.5,2,"{'learning_rate': 0.0003, 'max_depth': 3, 'min...",-0.853372,...,-0.583486,-0.528564,-0.561566,0.023752,7,-0.611322,-0.591983,-0.589153,-0.597486,0.009852
3,30.494691,12.099472,1.896525,0.019062,0.0003,3,0.5,3,"{'learning_rate': 0.0003, 'max_depth': 3, 'min...",-0.853154,...,-0.583361,-0.528436,-0.561439,0.023752,5,-0.611189,-0.591851,-0.589024,-0.597355,0.009850
4,22.140491,7.977848,1.845922,0.018726,0.003,2,0.5,2,"{'learning_rate': 0.003, 'max_depth': 2, 'min_...",-0.850228,...,-0.581365,-0.526342,-0.559405,0.023796,4,-0.609093,-0.589789,-0.587008,-0.595297,0.009821
5,25.655579,9.789339,1.883686,0.026059,0.003,2,0.5,3,"{'learning_rate': 0.003, 'max_depth': 2, 'min_...",-0.848453,...,-0.580184,-0.525110,-0.558203,0.023818,2,-0.607851,-0.588564,-0.585813,-0.594076,0.009805
6,25.398162,9.614106,1.881000,0.011382,0.003,3,0.5,2,"{'learning_rate': 0.003, 'max_depth': 3, 'min_...",-0.849455,...,-0.581244,-0.526277,-0.559287,0.023762,3,-0.608929,-0.589615,-0.586847,-0.595130,0.009822
7,30.455320,12.097248,1.920001,0.008961,0.003,3,0.5,3,"{'learning_rate': 0.003, 'max_depth': 3, 'min_...",-0.847298,...,-0.580004,-0.525012,-0.558026,0.023768,1,-0.607606,-0.588304,-0.585571,-0.593827,0.009807


In [18]:
resultados_GS['mean_test_neg_mean_squared_error']

0   -0.727560
1   -0.727392
2   -0.727503
3   -0.727307
4   -0.724562
5   -0.722912
6   -0.723995
7   -0.722065
Name: mean_test_neg_mean_squared_error, dtype: float64

In [19]:
resultados_GS['mean_test_neg_mean_absolute_error']

0   -0.561577
1   -0.561457
2   -0.561566
3   -0.561439
4   -0.559405
5   -0.558203
6   -0.559287
7   -0.558026
Name: mean_test_neg_mean_absolute_error, dtype: float64

In [20]:
#bst_grid.best_estimator_ #Mejores parámetros en general 
#bst_grid.best_score_ #Mejor MSE
bst_grid.best_index_ #De los modelos que se corrieron en que "lugar" esta el modelo de la tabla anterior
#bst_grid.scorer_ #Que medida de error se utiliza make_scorer(mean_squared_error, greater_is_better=False)

7

In [21]:
mejores_valores = dict(bst_grid.best_params_.items())
mejores_valores

{'learning_rate': 0.003,
 'max_depth': 3,
 'min_child_weight': 0.5,
 'n_estimators': 3}

In [22]:
MAE_entrenamiento = bst_grid.best_score_
MAE_entrenamiento

-0.5580263048215997

In [23]:
MSE_entrenamiento = (abs(-0.722065))
MSE_entrenamiento

0.722065

In [24]:
RMSE_entrenamiento = np.sqrt(abs(MSE_entrenamiento))
RMSE_entrenamiento

0.849744079120296

#### 9. Entrenar el modelo con la mejor combinación de parámetros obtenida

In [25]:
#reentrenamiento
import xgboost as xgb

In [26]:
xgbtrain = xgb.DMatrix(entrenamiento.iloc[:, entrenamiento.columns != 'ventas_por_mes'].values, 
                       entrenamiento.iloc[:, entrenamiento.columns == 'ventas_por_mes'].values)

param = {'max_depth': mejores_valores['max_depth'],
         'n_estimators': mejores_valores['n_estimators'],
         'min_child_weight': mejores_valores['min_child_weight'],
         'eta' : mejores_valores['learning_rate'], 
         'num_round' : 1000, 
         'seed' : 1,
         'silent' : 0,
         'eval_metric' : 'rmse'}
bst = xgb.train(param, xgbtrain)#1:46 - 1:47

[21:01:49] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[21:02:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:02:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:02:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:02:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:03:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:03:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:03:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[21:03:36] src/tree/updater_prune.cc:74: tre

#### 10. Preparar datos del conjunto de prueba para ser empleados en el modelo

In [27]:
#prueba remover la variable y 
prueba_x = prueba.drop('ventas_por_mes', axis = 1)
#prueba_x.head()

In [28]:
xgbpredict = xgb.DMatrix(prueba_x.values)

#### 11. Utilizar el modelo para predecir con conjunto de prueba

In [29]:
pred = bst.predict(xgbpredict)
#pred #Lista de predicción
#pd.Series(pred).describe() #Descripción de variables

#### 12. Revisar preedicciones

In [30]:
max(pred)

0.73149645

In [31]:
min(pred)

0.48569152

In [32]:
pred = np.clip(pred,0,200)

In [33]:
prediccion = pd.DataFrame(pred, columns = ['ventas_por_mes_pred'])
prediccion.head()

,ventas_por_mes_pred
0,0.485692
1,0.485692
2,0.485692
3,0.485692
4,0.485692


In [34]:
prediccion = prediccion.round()

In [36]:
predicciones_totales_entrenamiento = prediccion.sum()
predicciones_totales_entrenamiento

ventas_por_mes_pred    22724.0
dtype: float32

In [37]:
prueba_resultados = pd.concat([prueba.reset_index(drop=True), prediccion], axis = 1, ignore_index=False)

#### 13. Resultados conjunto de prueba

In [38]:
#calcular mse
prueba_resultados['error'] = ((prueba_resultados['ventas_por_mes'] - prueba_resultados['ventas_por_mes_pred']) ** 2) / 868595
#prueba_resultados.head()
MSE = prueba_resultados['error'].sum()
MSE

0.2278254

In [39]:
RMSE = MSE ** (1/2)
RMSE

0.4773105940721418

In [40]:
#Calcular MAE
prueba_resultados['error_MAE'] = abs(prueba_resultados['ventas_por_mes'] - prueba_resultados['ventas_por_mes_pred']) / 868595
prueba_resultados.head()
MAE = prueba_resultados['error_MAE'].sum()
MAE

0.06102498